In [ ]:
#start.py

import pygame, sys
from pygame.locals import *

WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)


pygame.init()
pygame.display.set_caption("!!! SNACKY !!!")
screen = pygame.display.set_mode((960, 640), 0,32)
background =  pygame.image.load('lantai1.png')

bit_font = pygame.font.Font(('vandervon.otf'), 120)
font = pygame.font.Font(('the.ttf'), 90)
choice = pygame.Rect(320,275,320,20)
arrow = pygame.image.load('arrow.png')

clock = pygame.time.Clock()

while True:
    screen.blit(background, [0, 0])
    clock.tick(10)
    
    for e in pygame.event.get():
        if e.type == QUIT:
            pygame.quit()
            sys.exit()
        elif e.type == pygame.KEYDOWN:
            if e.key == pygame.K_UP:
                if choice.top == 275:
                    choice.move_ip(0,150)
                else:
                    choice.move_ip(0,-150)
            elif e.key == pygame.K_DOWN:
                if choice.top == 425:
                    choice.move_ip(0,-150)
                else:
                    choice.move_ip(0,150)
            elif e.key == pygame.K_RETURN:
                if choice.top == 275:
                    import snacky
                elif choice.top == 425:
                    pygame.quit()
                    sys.exit()

                    
    screen.blit(bit_font.render("'SNACKY'",False,BLUE), (230,40))
    screen.blit(font.render("Play",False,BLUE), (370,250))
    screen.blit(font.render("Exit",False,BLUE), (385,400))
    screen.blit(arrow, choice)

    pygame.display.update()
    


In [ ]:
#snacky.py

import pygame, sys, random
pygame.init()

X = 960
Y = 640

BLACK = (0, 0, 0)
BACKGROUND = (178, 34, 34)
WHITE = (255, 255, 255)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
TRANSPARENT = (1, 2, 3)
bg = BACKGROUND

SIZE = 16
STEP = 2
fps = 100//STEP
n_invisible = SIZE//STEP-1
n_noCollision = SIZE//STEP * 3

screen = pygame.display.set_mode((960, 640), 0,32)
pygame.display.set_caption("Pemrograman Berorientasi Object (PyGame) Abad - Alvin")
clock = pygame.time.Clock()
font = pygame.font.Font(("the.ttf"), 30)


class Snake():
    def __init__(self, x, y, color, name):
        self.startx = x
        self.starty = y
        self.color = color
        self.name = name

    def reset(self):
        """ Each snake is build of several 'snakeElement's
    here the list is reset to only one element """
        self.snakeList = [SnakeElement(self.startx, self.starty, self.color)]
        self.length = 0
        self.dir = [False] * 4  # self.dir corresponds to down, up, left, right

    def grow(self):
        """ One Growth Cycle consist of a number of invisible
    elements, and one visible element (BLACK) """
        self.length += 1
        for _ in range(n_invisible):
            self.snakeList.append(SnakeElement(self.snakeList[-1].rect.centerx,
                                  self.snakeList[-1].rect.centery, TRANSPARENT, show=False))
        self.snakeList.append(SnakeElement(self.snakeList[-1].rect.centerx,
                              self.snakeList[-1].rect.centery, BLACK))

    def move(self):
        self.move_tail()
        self.move_head()

    def move_tail(self):        
        " tail moves one step forward "
        for i in range(len(self.snakeList)-1, 0, -1):
            self.snakeList[i].rect.center = self.snakeList[i-1].rect.center

    def move_head(self):
        " Head moves according to user input (self.dir) "
        head = self.snakeList[0]
        down, up, left, right = self.dir
        if down:
            head.rect.centery += STEP
        elif up:
            head.rect.centery -= STEP
        elif left:
            head.rect.centerx -= STEP
        elif right:
            head.rect.centerx += STEP
        
    def checkForCollisions(self):
        head = self.snakeList[0]
        ## --- check for collision with window --- ##
        if (head.rect.top < 0 or
            head.rect.bottom > Y or
            head.rect.left < 0 or
            head.rect.right > X):
            game.state = game.end
            return

        ## --- check for collision with self --- ##
        if (len(self.snakeList) > n_noCollision and
            head.rect.collidelist(self.snakeList[n_noCollision:]) > -1):
            game.state = game.end
            return

        ## --- check for hit with frog --- ##
        if frog.rect.colliderect(head):
            frog.reset()
            self.grow()

    def draw(self):
        for s in self.snakeList:
            s.draw()
        txt = font.render("%s: %d" %(self.name, self.length), 1, WHITE)
        screen.blit(txt, (10, 10))


class SnakeElement():
    def __init__(self, x, y, COLOR, rad=SIZE//2, light_pos=(0.35, -0.35), show=True):
        x0 = rad + 1
        y0 = rad + 1

        self.surf = pygame.surface.Surface((x0*2, y0*2))
        self.surf.fill(TRANSPARENT)
        self.surf.set_colorkey(TRANSPARENT)
        self.rect = self.surf.get_rect()
        self.rect.centerx = x
        self.rect.centery = y
        self.show = show

        if not show:
            return

        self.perimeter = []  # list of pixels that needs anti-aliasing

        x1 = x0 + int(light_pos[0] * rad)
        y1 = y0 + int(light_pos[1] * rad)

        for i in range(x0*2):
            for j in range(y0*2):
                d = ((i-x0)**2 + (j-y0)**2)**0.5
                if d >= rad+1:
                    continue
                else:
                    color = COLOR
                    # rate of color intensity change
                    d2 = ((i-x1)**2 + (j-y1)**2)**0.5 * 255 / rad
                    color = gradient(d2, color)

                    if d > rad:  # prepare for anti-aliasing, get the perimeter pixels
                        alfa = d-rad
                        color_rim = [c * (1-alfa) for c in color]
                        self.perimeter.append(((i, j), color_rim, alfa))
                        color = TRANSPARENT  # colorkey at rim

                    self.surf.set_at((i, j), color)

    def draw(self):
        if self.show is False:
            return
        
        screen.blit(self.surf, self.rect)

        # anti-aliasing
        for p in self.perimeter:
            x, y = p[0]
            x += self.rect.x
            y += self.rect.y
            if x < 0 or x >= X or y < 0 or y >= Y:
                continue
            color = p[1]
            alfa = p[2]
            bg = screen.get_at((x, y))
            color_aa = [rim + back * alfa for rim, back in zip(color, bg)]
            screen.set_at((x, y), color_aa)


class Frog():
    def __init__(self):
        self.surf = pygame.surface.Surface((SIZE, SIZE))
        self.surf.fill(bg)
        self.surf.blit(picFrog, [0, 0])
        self.rect = self.surf.get_rect()

    def draw(self):
        screen.blit(self.surf, self.rect)

    def reset(self):
        self.rect.centerx = random.randint(SIZE//2, X-SIZE//2)
        self.rect.centery = random.randint(30, Y-SIZE//2)
        while self.rect.collidelist(snake.snakeList) > -1:
            self.rect.centerx = random.randint(SIZE//2, X-SIZE//2)
            self.rect.centery = random.randint(30, Y-SIZE//2)


class GameState():
    def __init__(self):
        self.state = self.gameLoop

    def gameLoop(self):
        clock.tick(fps)  # game speed
        screen.fill(bg)

        self.eventLoop()
        snake.move()
        snake.checkForCollisions()

        frog.draw()
        snake.draw()
        pygame.display.flip()

    def end(self):
        againTxt()
        self.state = self.playAgain

    def stop(self):
        pygame.quit()
        sys.exit()

    def playAgain(self):
        pygame.time.wait(100)
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.state = self.stop
                return
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_n:
                    self.state = self.stop
                    return
                elif event.key == pygame.K_y:
                    snake.reset()
                    frog.reset()
                    self.state = self.gameLoop
                    return

    def eventLoop(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.state = self.stop
                return

            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_DOWN and not snake.dir[1]:
                    snake.dir = [True, False, False, False]
                elif event.key == pygame.K_UP and not snake.dir[0]:
                    snake.dir = [False, True, False, False]
                elif event.key == pygame.K_LEFT and not snake.dir[3]:
                    snake.dir = [False, False, True, False]
                elif event.key == pygame.K_RIGHT and not snake.dir[2]:
                    snake.dir = [False, False, False, True]                


def gradient(d2, color):
    r, g, b = color
    return [max(255-int(d2*(255-r)//150), 0),
            max(255-int(d2*(255-g)//150), 0),
            max(255-int(d2*(255-b )//150), 0)]


def againTxt():
    txt = font.render("Snake Dies", 1, RED)
    rct = txt.get_rect(center=[X//2-150, Y//2-40])
    screen.blit(txt, rct)

    txt = font.render("Main Lagi? (Y / N)?", 1, WHITE)
    rct = txt.get_rect(center=[X//2-150, Y//2])
    screen.blit(txt, rct)

    pygame.display.flip()


picFrog = pygame.image.load("frog.png").convert()
picFrog = pygame.transform.scale(picFrog, (SIZE, SIZE))
picFrog.set_colorkey(WHITE)

snake = Snake(X//2, Y//3, YELLOW, 'Yellow Player ')
frog = Frog()
game = GameState()

snake.reset()
frog.reset()

while True:
    game.state()

